In [1]:
import json
import os
import numpy as np
from pycocotools.coco import COCO
import open3d as o3d
import cv2
from sklearn.cluster import DBSCAN
from skimage.morphology import skeletonize
from sklearn.neighbors import NearestNeighbors
import scipy.signal as sps
from scipy.ndimage import gaussian_filter, binary_erosion

import matplotlib.pyplot as plt
%matplotlib inline

import kneed
from kneebow.rotor import Rotor

import pandas as pd

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
jsonPath = r"E:\RootAI\plantannotated\Onion1\Onion1.json"


coco_annotations = COCO(annotation_file = jsonPath)
cat_ids = coco_annotations.getCatIds()
print('COCO category ids:')
print(cat_ids)
cats = coco_annotations.loadCats(cat_ids)
names=[cat['name'] for cat in cats]
print('COCO categories: \n{}\n'.format(' '.join(names)))
image_ids = coco_annotations.getImgIds(catIds = cat_ids)
print("num images:", len(image_ids))

with open(jsonPath) as jsonFile:
    cocoAnns = json.load(jsonFile)
jsonFile.close()

images = cocoAnns['images']
annotations = cocoAnns['annotations']

print("num annotations:", len(annotations))

imageList = [[None]*2 for _ in range(len(images))]
for im in range(len(images)):
    fileName = images[im]['file_name']
    index = images[im]['id']
    imageList[im][0] = fileName
    imageList[im][1] = index
    
imageList = np.array(imageList)

loading annotations into memory...
Done (t=1.58s)
creating index...
index created!
COCO category ids:
[1]
COCO categories: 
root

num images: 3196
num annotations: 32899


In [3]:
jsonPath2 = r"E:\RootAI\plantannotated\predictions\100\3view\Onion1.json"


coco_annotations2 = COCO(annotation_file = jsonPath2)
cat_ids = coco_annotations2.getCatIds()
print('COCO category ids:')
print(cat_ids)
cats = coco_annotations2.loadCats(cat_ids)
names=[cat['name'] for cat in cats]
print('COCO categories: \n{}\n'.format(' '.join(names)))
image_ids = coco_annotations2.getImgIds(catIds = cat_ids)
print("num images:", len(image_ids))

with open(jsonPath2) as jsonFile2:
    cocoAnns2 = json.load(jsonFile2)
jsonFile2.close()

images = cocoAnns2['images']
annotations = cocoAnns2['annotations']

print("num annotations:", len(annotations))

imageList2 = [[None]*2 for _ in range(len(images))]
for im in range(len(images)):
    fileName = images[im]['file_name']
    index = images[im]['id']
    imageList2[im][0] = fileName
    imageList2[im][1] = index
    
imageList2 = np.array(imageList2)
imageNames2 = imageList2[:,0]

loading annotations into memory...
Done (t=1.03s)
creating index...
index created!
COCO category ids:
[1]
COCO categories: 
root

num images: 3497
num annotations: 19021


In [4]:
def getIOU(mask_gt, mask_pred):
    intersection = np.logical_and(mask_gt,mask_pred)
    union = np.logical_or(mask_gt,mask_pred)
    iou = np.sum(intersection)/np.sum(union)
    return iou

In [5]:
def get_pr_re(mask_gt, mask_pred):
    
    mask_gt[mask_gt==1] = 2
    
    added = np.add(mask_gt, mask_pred)
    
    tn = np.sum(added == 0)
    tp = np.sum(added == 3)
    fn = np.sum(added == 2)
    fp = np.sum(added == 1)
    
    return tn, tp, fn, fp

In [6]:
def get_np(inList, coco_anns):

    point_cloud_x = []
    point_cloud_y = []
    point_cloud_z = []

    imageNames = inList[:,0]

    for image in imageNames:
        #print(image)
        name = image.split(".")[0]
    
        split = name.split("_",2)
        axis = split[2]
        num = int(split[1][-3:])
    
        z = np.where(inList[:,0]==image)[0][0]

        imId= int(inList[z,1])
        anns_ids = coco_anns.getAnnIds(imgIds=imId, catIds=[], iscrowd=None)
        anns = coco_anns.loadAnns(anns_ids)
        
        if (axis == 'x'):
            mask = np.full((920,1998),False)
        if (axis == 'y'):
            mask = np.full((920,1998),False)
        if (axis == 'z'):
            mask = np.full((920,920),False)
       
    
        if(len(anns)>0):
    
            mask = coco_anns.annToMask(anns[0])
        
            for i in range(1,len(anns)):
                mask += coco_anns.annToMask(anns[i])
    
    
        if (axis == 'x'):
            point_cloud_x.append(mask)
        if (axis == 'y'):
            point_cloud_y.append(mask)       
        if (axis == 'z'):
            point_cloud_z.append(mask) 
  
    scan_x = np.array(point_cloud_x)
    scan_y = np.array(point_cloud_y) 
    scan_z = np.array(point_cloud_z) 
    
    scan_y = np.fliplr(scan_y)
    scan_y = np.rot90(scan_y)
    
    scan_z = np.swapaxes(scan_z,0,2)
    scan_z = np.fliplr(scan_z)
    scan_z = np.rot90(scan_z)
    
    return scan_x, scan_y, scan_z

In [7]:
scan_x_gt, scan_y_gt, scan_z_gt = get_np(imageList, coco_annotations)
scan_x_pred, scan_y_pred, scan_z_pred = get_np(imageList2, coco_annotations2)

In [9]:
result_xy = np.add(scan_x_pred, scan_y_pred)
result_xz = np.add(scan_x_pred, scan_z_pred)
result_yz = np.add(scan_y_pred, scan_z_pred)
result_xyz = np.add(result_xy, scan_z_pred)

result_xy = result_xy.clip(min = 0 , max = 1)
result_xz = result_xz.clip(min = 0 , max = 1)
result_yz = result_yz.clip(min = 0 , max = 1)
result_xyz = result_xyz.clip(min = 0 , max = 1)

result_x = scan_x_pred.clip(min = 0 , max = 1)
result_y = scan_y_pred.clip(min = 0 , max = 1)
result_z = scan_z_pred.clip(min = 0 , max = 1)

gt_xy = np.add(scan_x_gt, scan_y_gt)
gt_xyz = np.add(gt_xy, scan_z_gt)
gt_xyz = gt_xyz.clip(min = 0 , max = 1)

In [12]:
#check point clouds if you want
indeces = np.nonzero(scan_x_pred)
indeces = np.asarray(indeces)
indeces = np.swapaxes(indeces, 0, 1)

indeces2 = np.nonzero(scan_y_pred)
indeces2 = np.asarray(indeces2)
indeces2 = np.swapaxes(indeces2, 0, 1)

indeces3 = np.nonzero(scan_z_pred)
indeces3 = np.asarray(indeces3)
indeces3 = np.swapaxes(indeces3, 0, 1)

pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(indeces)
pcd.paint_uniform_color([1, 0.706, 0])

pcd2 = o3d.geometry.PointCloud()
pcd2.points = o3d.utility.Vector3dVector(indeces2)
pcd2.paint_uniform_color([0.1, 0.1, 0.7])

pcd3 = o3d.geometry.PointCloud()
pcd3.points = o3d.utility.Vector3dVector(indeces3)
pcd3.paint_uniform_color([0.1, 0.9, 0.1])

PointCloud with 67918795 points.

In [13]:
o3d.visualization.draw_geometries([pcd,pcd2,pcd3])

In [14]:
cut_top = 288
cut_bottom = 480

z_total = result_xyz.shape[2]

result_xyz_cut = result_xyz[:,:,cut_bottom:z_total-cut_top]

indeces = np.nonzero(result_xyz_cut)
indeces = np.asarray(indeces)
indeces = np.swapaxes(indeces, 0, 1)

pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(indeces)
pcd.paint_uniform_color([1, 0.706, 0])

o3d.visualization.draw_geometries([pcd])

In [15]:
def mask_to_contour(mask):
    contours, _ = cv2.findContours(
        mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE
    )
    return contours

In [16]:
def moving_average(x, w):
    return np.convolve(x, np.ones(w), 'valid') / w

In [17]:
def array2pc(array):
    indeces = np.nonzero(array)
    indeces = np.asarray(indeces)
    indeces = np.swapaxes(indeces, 0, 1)
    outpcd = o3d.geometry.PointCloud()
    outpcd.points = o3d.utility.Vector3dVector(indeces) 
    
    return outpcd

In [18]:
def pc2array(pointcloud, array):
    out = np.zeros(array.shape,dtype=np.uint8)
    xyz = np.asarray(pointcloud.points).astype(int)
    
    for pt in xyz:
        out[int(pt[0]),int(pt[1]),int(pt[2])] = 1
    
    return out

In [19]:
def dbscan(pointcloud,dist,neighbors,array):
    
    out = np.zeros(array.shape,dtype=np.uint8)
    
    with o3d.utility.VerbosityContextManager(
        o3d.utility.VerbosityLevel.Debug) as cm:
        labels = np.array(
        pointcloud.cluster_dbscan(eps=eps, min_points=6, print_progress=True))

    xyz = np.asarray(downpcd.points).astype(int)
    
    unique, counts = np.unique(labels, return_counts = True)
    sorted_counts = np.sort(counts, axis = 0)
    i = np.arange(len(sorted_counts))
    knee = kneed.KneeLocator(i, sorted_counts, curve='convex', direction='increasing', online = True)
    indeces = np.where(counts > sorted_counts[knee.knee])
    selected_labels = unique[indeces]
    selected_labels = np.array(selected_labels)  
    selected_labels = selected_labels[selected_labels != -1]
    
    for i in range(len(labels)):
        label = labels[i]
        for j in range(len(selected_labels)):
            if(label == selected_labels[j]):
                down_indx = xyz[i]
                out[down_indx[0],down_indx[1],down_indx[2]] = 1
    
  
    return out

In [20]:
def findeps(array):
    X = np.argwhere(array)
    neigh = NearestNeighbors(n_neighbors=6)
    nbrs = neigh.fit(X)
    distances, indeces = nbrs.kneighbors(X)
    distances = np.sort(distances, axis=0)
    distances = distances[:,1]
    convolved = moving_average(distances,5)
    i = np.arange(len(convolved))
    knee = kneed.KneeLocator(i, convolved, curve='convex', direction='increasing', online = False)
    x = distances[knee.knee]
    
    return x

In [21]:
def dilate_erode(array):
    kernel = np.ones((5, 5), np.uint8)
    out = np.zeros(array.shape, dtype=np.uint8)
    
    for i in range(array.shape[2]):
        mask = array[:,:,i]
        mask_dilated = cv2.dilate(mask, kernel, iterations=1)
        mask_eroded = cv2.erode(mask_dilated, kernel, iterations=1)
        
        out[:,:,i] = mask_eroded
        
    for i in range(array.shape[0]):
        mask = out[i,:,:]
        mask_dilated = cv2.dilate(mask, kernel, iterations=1)
        mask_eroded = cv2.erode(mask_dilated, kernel, iterations=1)
        
        out[i,:,:] = mask_eroded
        
    out = np.clip(out,0,1)
    
    return out

In [22]:
results = [result_x,result_y,result_z,result_xy,result_xz,result_yz,result_xyz]
combos = ['x','y','z','xy','xz','yz','xyz']

In [36]:
cut_top = 288
cut_bottom = 480

z_total = result_xyz.shape[2]

gt_xyz_cut = gt_xyz[:,:,cut_bottom:z_total-cut_top]

metrics = []
for k in range(len(results)):
    
    result_pred = results[k]
    
    result_pred = result_pred[:,:,cut_bottom:z_total-cut_top]
    
    result_pred_pc = array2pc(result_pred)
    downpcd = result_pred_pc.voxel_down_sample(voxel_size=1.5)
    downpcd_np = pc2array(downpcd, result_pred)
    eps = findeps(downpcd_np)
    filtered = dbscan(downpcd, eps, 6, result_pred)
    filtered_restored = dilate_erode(filtered)

    iou = getIOU(gt_xyz_cut,filtered_restored)
    tn, tp, fn, fp = get_pr_re(gt_xyz_cut,filtered_restored)
    
    precision = tp / (tp + fp)
    recall = tp / (tp + fn)
    accuracy = (tn + tp) / (tn + tp + fn + tp)
    dice = 2*(precision*recall)/(precision + recall)
            
    print([combos[k],precision,recall,dice,iou])
        
    metrics.append([combos[k], cut_bottom, cut_top, precision,recall,dice,iou])

[Open3D DEBUG] Precompute neighbors.
[Open3D DEBUG] Done Precompute neighbors.
[Open3D DEBUG] Compute Clusters
[Open3D DEBUG] Done Compute Clusters: 368
['x', 0.8189250820597571, 0.7041595807183799, 0.7572185007380154, 0.6092933481772005]
[Open3D DEBUG] Precompute neighbors.
[Open3D DEBUG] Done Precompute neighbors.
[Open3D DEBUG] Compute Clusters
[Open3D DEBUG] Done Compute Clusters: 358
['y', 0.7125219734183246, 0.5237457805901634, 0.6037209592268665, 0.4323784441343331]
[Open3D DEBUG] Precompute neighbors.
[Open3D DEBUG] Done Precompute neighbors.
[Open3D DEBUG] Compute Clusters
[Open3D DEBUG] Done Compute Clusters: 386
['z', 0.8380104704288137, 0.7427500182453871, 0.7875099316568405, 0.6494980472153105]
[Open3D DEBUG] Precompute neighbors.
[Open3D DEBUG] Done Precompute neighbors.
[Open3D DEBUG] Compute Clusters
[Open3D DEBUG] Done Compute Clusters: 378
['xy', 0.6860635472455773, 0.8478841389043632, 0.7584383812332032, 0.6108745387816801]
[Open3D DEBUG] Precompute neighbors.
[Open3

In [38]:
path = r'E:\RootAI\plantannotated\predictions\100\3view\outputMetricsDBSCAN_cut.xlsx'
index = ['Onion1' for n in range(len(metrics))]
df = pd.DataFrame(metrics,index = index,columns = ['view','cut_bottom','cut_top' ,'precision','recall','dice','iou'])
print(df)

with pd.ExcelWriter(path, mode = 'a', if_sheet_exists = 'new') as writer: 
    df.to_excel(writer)

       view  cut_bottom  cut_top  precision    recall      dice       iou
Onion1    x         480      288   0.818925  0.704160  0.757219  0.609293
Onion1    y         480      288   0.712522  0.523746  0.603721  0.432378
Onion1    z         480      288   0.838010  0.742750  0.787510  0.649498
Onion1   xy         480      288   0.686064  0.847884  0.758438  0.610875
Onion1   xz         480      288   0.726658  0.872903  0.793095  0.657132
Onion1   yz         480      288   0.698868  0.849988  0.767056  0.622133
Onion1  xyz         480      288   0.654004  0.892726  0.754943  0.606352


In [24]:
result_pred = result_xyz
    
cut_top = 288
cut_bottom = 480

z_total = result_xyz.shape[2]

gt_xyz_cut = gt_xyz[:,:,cut_bottom:z_total-cut_top]
result_pred = result_pred[:,:,cut_bottom:z_total-cut_top]

result_pred_pc = array2pc(result_pred)
downpcd = result_pred_pc.voxel_down_sample(voxel_size=1.5)
downpcd_np = pc2array(downpcd, result_pred)
eps = findeps(downpcd_np)
filtered = dbscan(downpcd, eps, 6, result_pred)
filtered_restored = dilate_erode(filtered)

iou = getIOU(gt_xyz_cut,filtered_restored)
tn, tp, fn, fp = get_pr_re(gt_xyz_cut,filtered_restored)
    
precision = tp / (tp + fp)
recall = tp / (tp + fn)
accuracy = (tn + tp) / (tn + tp + fn + tp)
dice = 2*(precision*recall)/(precision + recall)
            
print([precision,recall,dice,iou])

[Open3D DEBUG] Precompute neighbors.
[Open3D DEBUG] Done Precompute neighbors.
[Open3D DEBUG] Compute Clusters
[Open3D DEBUG] Done Compute Clusters: 471
[0.6540037029778643, 0.8927263123683773, 0.7549427608463344, 0.6063518504253753]


In [25]:
indeces = np.nonzero(filtered_restored)
indeces = np.asarray(indeces)
indeces = np.swapaxes(indeces, 0, 1)

pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(indeces)
pcd.paint_uniform_color([1, 0.706, 0])

o3d.visualization.draw_geometries([pcd])